# Waldemar Chang - Assignment 4: Constructing the Inference Service
## EN.705.603.82.FA24 Creating AI-Enabled Systems
### Task 5
#### Using a notebook called model_performance.ipynb, analyze the performance of the two models provided to you. Use the TechTrack Dataset to perform this analysis. Finally, provide a thorough argument on why you favor one model over another.

In [ ]:
import os
import random
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from nms import filter
from object_detection import Model, draw_bboxes
from helper import calculate_iou, calculate_pr, calculate_ap, calculate_map, calculate_11pi, calculate_precision_recall_f1, calculate_specificity, get_ground_truth, denormalize

In [ ]:
# Define path to folder containing images and labels
folder_path = r"C:\Users\walde\techtrack\notebooks\logistics"

# Initialize models
m1 = Model('model1.cfg', 'model1.weights')
m2 = Model('model2.cfg', 'model2.weights')

In [ ]:
# Get all image file names
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

# Randomly select 10 images
random.seed(42)
sample_size = 10
sampled_images = random.sample(image_files, sample_size)

#### 100 Images

In [ ]:
# Loop through all files in the directory
for file_name in sampled_images:
    if file_name.endswith('.jpg'):
        # Get base name without the extension
        base_name = os.path.splitext(file_name)[0]
        
        # Define corresponding text file path
        text_file = os.path.join(folder_path, f"{base_name}.txt")
        
        # Check if corresponding text file exists
        if os.path.exists(text_file):
            # Read the image
            image_path = os.path.join(folder_path, file_name)
            image = cv.imread(image_path)
            
            # Read ground truth data
            ground_truth = get_ground_truth(text_file)
            gt_bboxes = [bbox for _, bbox in ground_truth]
            gt_class_ids = [class_id for class_id, _ in ground_truth]
            
            # Perform first model's NMS filtered prediction here
            m1_pp_frame, m1_og_frame = m1.preprocess(image)
            m1_pred_bboxes, m1_pred_class_ids, m1_pred_scores = m1.predict(m1_pp_frame)
            m1_post_bboxes, m1_post_class_ids, m1_post_scores = m1.post_process(m1_pred_bboxes, m1_pred_class_ids, m1_pred_scores, m1_og_frame)
            m1_nms_bboxes, m1_nms_class_ids, m1_nms_labels, m1_nms_scores = filter(m1_post_bboxes, m1_post_class_ids, m1_post_scores, 0.5, 0.4)

            # Perform second model's NMS filtered prediction here
            m2_pp_frame, m2_og_frame = m2.preprocess(image)
            m2_pred_bboxes, m2_pred_class_ids, m2_pred_scores = m2.predict(m2_pp_frame)
            m2_post_bboxes, m2_post_class_ids, m2_post_scores = m2.post_process(m2_pred_bboxes, m2_pred_class_ids, m2_pred_scores, m2_og_frame)
            m2_nms_bboxes, m2_nms_class_ids, m2_nms_labels, m2_nms_scores = filter(m2_post_bboxes, m2_post_class_ids, m2_post_scores, 0.5, 0.4)
            
            # Calculate performance metrics
            # Calculate metrics for Model 1
            precision_m1, recall_m1 = calculate_pr(m1_nms_bboxes, m1_nms_class_ids, m1_nms_scores, gt_bboxes, gt_class_ids)
            ap_m1 = calculate_ap(precision_m1, recall_m1)
            aps_model1.append(ap_m1)

            # Calculate metrics for Model 2
            precision_m2, recall_m2 = calculate_pr(m2_nms_bboxes, m2_nms_class_ids, m2_nms_scores, gt_bboxes, gt_class_ids)
            ap_m2 = calculate_ap(precision_m2, recall_m2)
            aps_model2.append(ap_m2)
            
            # Visualize results
            # Draw ground truth and predicted bounding boxes on image for comparison
            image_gt = image.copy()
            image_gt = draw_bboxes(image_gt, gt_bboxes, gt_class_ids)
            
            # Draw predicted boxes for Model 1
            image_m1 = image.copy()
            image_m1 = draw_bboxes(image_m1, m1_nms_bboxes, m1_nms_class_ids, m1_nms_scores)
            #image_m1 = draw_bboxes(image_m1, m1_post_bboxes, m1_post_class_ids, m1_post_scores)
            
            # Draw predicted boxes for Model 2
            image_m2 = image.copy()
            image_m2 = draw_bboxes(image_m2, m2_nms_bboxes, m2_nms_class_ids, m2_nms_scores)
            #image_m2 = draw_bboxes(image_m2, m2_post_bboxes, m2_post_class_ids, m2_post_scores)
            
            # Display images side by side
            plt.figure(figsize=(15, 10))
            plt.subplot(1, 3, 1)
            plt.imshow(cv.cvtColor(image_gt, cv.COLOR_BGR2RGB))
            plt.title("Ground Truth")
            plt.axis('off')

            plt.subplot(1, 3, 2)
            plt.imshow(cv.cvtColor(image_m1, cv.COLOR_BGR2RGB))
            plt.title("Model 1 Predictions")
            plt.axis('off')

            plt.subplot(1, 3, 3)
            plt.imshow(cv.cvtColor(image_m2, cv.COLOR_BGR2RGB))
            plt.title("Model 2 Predictions")
            plt.axis('off')

            plt.suptitle(f"Comparison for {file_name}", fontsize=16)
            plt.show()
            
        else:
            print(f"No corresponding text file found for {file_name}")
            
# Calculate mAP for both models
mAP_model1 = calculate_map(aps_model1)
mAP_model2 = calculate_map(aps_model2)

print(f"Model 1 mAP: {mAP_model1:.4f}")
print(f"Model 2 mAP: {mAP_model2:.4f}")

In [ ]:
print(aps_model1)

In [ ]:
print(aps_model2)